# ETL sobre 'users_items.json'

Importamos librerias 

In [1]:
from pandas import json_normalize
import pandas as pd 
import json 
import ast 

Guardamos la ruta de acceso  dentro de una variable 


In [2]:
path_items = r'C:\Users\Usuario\Desktop\proyecto steam\Dataset\australian_users_items.json'

Extraemos datos de "australian_users_items.json" , creamos un Dataframe del archivo original.\
Creamos una copia del Dataframe para sus transformaciones.


In [3]:
lista_items = []
with open(path_items, encoding='utf-8') as file:
    for line in file.readlines():
        lista_items.append(ast.literal_eval(line))

 
df_Items= pd.DataFrame(lista_items)

df_Items_clean = df_Items.copy()

Controlamos las primeras 10 filas del dataframe

In [4]:
df_Items_clean.head(10)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
5,MinxIsBetterThanPotatoes,371,76561198004744620,http://steamcommunity.com/id/MinxIsBetterThanP...,"[{'item_id': '50', 'item_name': 'Half-Life: Op..."
6,NitemarePK,304,76561197990951820,http://steamcommunity.com/id/NitemarePK,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
7,themanwich,258,76561198035296505,http://steamcommunity.com/id/themanwich,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
8,maplemage,629,76561198026584251,http://steamcommunity.com/id/maplemage,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
9,Wackky,0,76561198039117046,http://steamcommunity.com/id/Wackky,[]


### En la columna 'items' decidimos desanidar los diccionarios, convirtiendo la clave en columna y el valor en una nueva fila por registro

transformamos el DataFrame al descomponer listas en la columna 'items', lo que resulta en una representación más granular de los datos de los elementos.

In [5]:
df_items_expanded = df_Items_clean.explode('items')

optimizamos la estructura de índice del DataFrame después de explotar listas, garantizando un esquema de indexación claro y consistente para el análisis de datos posterior.

In [6]:
df_items_expanded.reset_index(drop=True, inplace=True)

Normalizamos, lo que permite un análisis y una manipulación de datos más efectivos.

In [7]:
df_items_normalized = json_normalize(df_items_expanded['items'])


Integramos los datos originales de df_Items_clean (excluyendo la columna 'items' explotada) con los datos normalizados extraídos de los objetos JSON.

In [8]:
df_items_final = pd.concat([df_items_expanded.drop(columns=['items']), df_items_normalized], axis=1)

Eliminamos columnas que no son utiles para el proyecto.

In [9]:
# Lista de columnas a eliminar
columns_to_drop = ['steam_id','user_url','item_name','playtime_2weeks'] 

# Eliminar las columnas especificadas
df_Items_clean = df_items_final.drop(columns=columns_to_drop, errors='ignore')

Exportamos a .parquet

In [10]:
df_Items_clean.to_parquet(r'C:\Users\Usuario\Desktop\proyecto steam\Data Exportada\user_items_clean.parquet', index=False)